In [1]:
import torch
import scipy
import pandas as pd
import numpy as np
from typing import Tuple
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.models import resnet18
import requests
device = torch.device('cpu' if torch.backends.mps.is_available() else 'cpu')

In [2]:
class TaskDataset(Dataset):
    def __init__(self, transform=None):
        self.ids = []
        self.imgs = []
        self.labels = []
        self.transform = transform
    def __getitem__(self, index) -> Tuple[int, torch.Tensor, int]:
        id_ = self.ids[index]
        img = self.imgs[index]
        if not self.transform is None:
            img = self.transform(img)
        label = self.labels[index]
        return id_, img, label
    def __len__(self):
        return len(self.ids)

In [3]:
class MembershipDataset(TaskDataset):
    def __init__(self, transform=None):
        super().__init__(transform)
        self.membership = []
    def __getitem__(self, index) -> Tuple[int, torch.Tensor, int, int]:
        id_, img, label = super().__getitem__(index)
        return id_, img, label, self.membership[index]

In [4]:
transform_00 = transforms.Compose([
    transforms.Normalize(mean=[0.2980, 0.2962, 0.2987], std=[0.2886, 0.2875, 0.2889]),  # Normalize with mean and std
])
transform_01 = transforms.Compose([
    transforms.Normalize(mean=[0.2980, 0.2962, 0.2987], std=[0.2886, 0.2875, 0.2889]),  # Normalize with mean and std
    transforms.RandomHorizontalFlip(p=1),  # Apply horizontal flip
])
transform_10 = transforms.Compose([
    transforms.Normalize(mean=[0.2980, 0.2962, 0.2987], std=[0.2886, 0.2875, 0.2889]),  # Normalize with mean and std
    transforms.RandomVerticalFlip(p=1),    # Apply vertical flip
])
transform_11 = transforms.Compose([
    transforms.Normalize(mean=[0.2980, 0.2962, 0.2987], std=[0.2886, 0.2875, 0.2889]),  # Normalize with mean and std
    transforms.RandomHorizontalFlip(p=1),  # Apply horizontal flip
    transforms.RandomVerticalFlip(p=1),    # Apply vertical flip
])

In [5]:
priv_dataset = torch.load('out/data/priv.pt')
pub_dataset = torch.load('out/data/pub.pt')
priv_in_scores = torch.load('splits/in_scores.pt')
priv_out_scores = torch.load('splits/out_scores.pt')

/var/folders/rq/9d8zfn5n4_503rc3q_gkyglw0000gr/T/ipykernel_33357/1679048514.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  priv_dataset = torch.load('out/data/priv.pt')

In [6]:
priv_loader = DataLoader(priv_dataset, batch_size=16, shuffle=True)
pub_loader = DataLoader(pub_dataset, batch_size=16, shuffle=True)

In [7]:
main_model = resnet18(pretrained=False)
main_model.fc = torch.nn.Linear(512, 44)
ckpt = torch.load("out/models/attack_model.pt", map_location=device)
main_model.load_state_dict(ckpt)
main_model.eval()

/opt/anaconda3/envs/pathwaysasuke/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/anaconda3/envs/pathwaysasuke/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/var/folders/rq/9d8zfn5n4_503rc3q_gkyglw0000gr/T/ipykernel_33357/2896128036.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future relea

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [8]:
def likelihood_ratio(score, mean_1, std_1, mean_2, std_2):
    #print(scipy.stats.norm(mean_1, std_1).pdf(score), scipy.stats.norm(mean_2, std_2).pdf(score))
    return scipy.stats.norm(mean_1, std_1).pdf(score) / scipy.stats.norm(mean_2, std_2).pdf(score)

In [ ]:
priv_lira = dict()
for i, (id_, img, label) in enumerate(zip(priv_dataset.ids, priv_dataset.imgs, priv_dataset.labels)):
    in_score_id = np.array(priv_in_scores[id_])
    out_score_id = np.array(priv_out_scores[id_])
    in_score_id_ = list()
    out_score_id_ = list()
    for score in in_score_id:
        if np.isfinite(score):
            in_score_id_.append(score)
        elif score == np.float64('inf'):
            in_score_id_.append(10)
        elif score == -np.float64('inf'):
            in_score_id_.append(-10)
    for score in out_score_id:
        if np.isfinite(score):
            out_score_id_.append(score)
        elif score == np.float64('inf'):
            out_score_id_.append(10)
        elif score == -np.float64('inf'):
            out_score_id_.append(-10)
    in_score_id_ = np.array(in_score_id_)
    out_score_id_ = np.array(out_score_id_)
    #print(f'{id_}: {(in_score_id_.mean(), in_score_id_.std())}, {(out_score_id_.mean(), out_score_id_.std())}')
    img_00 = transform_00(img).unsqueeze(0)
    img_01 = transform_01(img).unsqueeze(0)
    img_10 = transform_10(img).unsqueeze(0)
    img_11 = transform_11(img).unsqueeze(0)
    imgs_ = torch.cat((img_00, img_01, img_10, img_11), dim=0)
    outputs = main_model(imgs_)
    raw_scores = torch.nn.functional.softmax(outputs, dim=1)
    #print(raw_scores.shape)
    confidence_score = raw_scores[:, label]
    #print(confidence_score.shape)
    logit_score = torch.log(confidence_score/(1-confidence_score))
    likelihood = list()
    #print(logit_score)
    for j in range(4):
        if logit_score[j].detach().numpy() == np.float64('inf'):
            likelihood.append(2)
        elif logit_score[j].detach().numpy() == -np.float64('inf'):
            likelihood.append(0.5)
        else:
            likelihood.append(likelihood_ratio(logit_score[j].detach().numpy(), in_score_id_.mean(), in_score_id_.std(), out_score_id_.mean(), out_score_id_.std()))
    #print(likelihood)
    priv_lira[id_] = np.sqrt(np.sqrt(np.array(likelihood).prod()))
    if (i+1) % 1000 == 0:
        print(f"Processed {i+1} images")

Processed 1000 images
Processed 2000 images
Processed 3000 images
Processed 4000 images
Processed 5000 images
Processed 6000 images
Processed 7000 images
Processed 8000 images
Processed 9000 images
Processed 10000 images
Processed 11000 images
Processed 12000 images
Processed 13000 images
Processed 14000 images
Processed 15000 images
Processed 16000 images
Processed 17000 images
Processed 18000 images
Processed 19000 images
Processed 20000 images


In [10]:
priv_lira

{55061: 0.32095781942029905,
 67669: 0.6371374765807509,
 228925: 1.5625336552536617,
 4248: 1.3734291535492789,
 109584: 0.8583141009866676,
 117235: 2.3369866164869193,
 71898: 1.0827000145351675,
 109497: 0.9083399134487076,
 117936: 1.0286559514436884,
 116979: 1.2194198907553844,
 252165: 2.2900885040805803,
 44279: 1.4841926046957497,
 97390: 1.1811096906942926,
 60210: 2.698722569180098,
 27707: 0.40784918540188614,
 214933: 2.316254536785116,
 64394: 0.179309306065946,
 209999: 0.25462152497257373,
 47952: 0.05265649058107078,
 232835: 2.410142932935394,
 113706: 0.9912597268064164,
 247669: 0.5029063030622862,
 37143: 0.9800493262208796,
 222156: 1.0443013125265714,
 28522: 0.8861593590315976,
 110978: 0.25290274183067063,
 147994: 0.902343315595407,
 9344: 0.7763590446703948,
 159763: 1.7228387099594913,
 207324: 2.5607671067282696,
 197822: 6.3659699282264635,
 277575: 0.33827822348699316,
 86673: 0.829180968101448,
 135646: 1.0116985719923746,
 26928: 0.14783262698907157,
 

In [11]:
df = pd.DataFrame(
    {
        "ids": priv_lira.keys(),
        "score": priv_lira.values(),
    }
)
df.to_csv("lira_16.csv", index=None)

In [12]:
response = requests.post("http://35.239.75.232:9090/mia", files={"file": open("lira_16.csv", "rb")}, headers={"token": "13301858"})
print(response.json())

{'TPR@FPR=0.05': 0.07633333333333334, 'AUC': 0.5396852777777777}
